In [1]:
import scrapy
import numpy as np
from scrapy.crawler import CrawlerProcess
import re
import json

In [2]:
class Diputados(scrapy.Spider):
    name = "spider_diputados"
    
    def start_requests(self):
        links = [urlCurr + i for i in terminaciones]
        for link in links:
            yield scrapy.Request( url = link, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, 
                              callback = self.parse )
    
    def parse(self,response):
        nombre = response.css('span.Estilo67::text').extract_first()
        nombre = limpiaText(nombre) # cambié esto
        idDipu = ''.join([c for c in nombre[1:] if c.isupper()])
        cumple = response.css('td.textoNegro::text').extract()[5]
        curp = creaCurp(nombre,cumple)
        dipus[curp]={}
        dipus.get(curp,{})['CURP']=curp # Agregué esto para 
        dipus.get(curp,{})['Nombre']=nombre # cambié aquí
        dipus.get(curp,{})['TipoElección']=response.css('td.textoNegro::text').extract()[0]
        dipus.get(curp,{})['Entidad']=response.css('td.textoNegro::text').extract()[1]
        parti=response.xpath('//tr/td/table//td//img/@src').extract()[1]
        dipus.get(curp,{})['Partido']=obtienePartido(parti)
        distri1 =response.css('td.textoNegro::text').extract()[2]
        distri2 =response.css('td.textoNegro::text').extract()[4]  # PARTE RARA
        dipus.get(curp,{})['Distrito'] = distri1 +""+ distri2
        dipus.get(curp,{})['Correo']=response.css('a.linkNegroSin::text').extract()[0]
        dipus.get(curp,{})['Onomástico']=limpiaText(cumple)
        supl=response.css('span.Estilo67::text').extract()[6]
        dipus.get(curp,{})['Suplente']=limpiaText(supl)
        dipus.get(curp,{})['Comisiones']=response.xpath('//a[contains(@href, "integrantes_de_comision")]/text()').getall()
        dipus.get(curp,{})['Último grado de estudios']=response.xpath('//td[@class="textoNegro"]/text()').extract_first()
        link_asis=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[2]
        link_votaciones=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[3]
        #yield response.follow(url=link_asis, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
        #                     callback=self.parse2)
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse4)
    
    def parse2(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse3)
    
    def parse3(self,response):
        curp = response.meta['curp']
        prueba = response.xpath('//tr/td/div/font/text()').extract()
        dias = list(filter(lambda a: (a == 'A' or a =='AC'), prueba))
        dipus.get(curp,{})['Asistencias']=dipus.get(curp,{}).get('Asistencias',0) + len(dias)

    def parse4(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse5)
                
    def parse5(self,response):
        curp = response.meta['curp']
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        aux2={}
        for i in range(1,len(aux),4):
            votos = {}
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i]
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]
            if(votacion in por_votaciones.keys()):
                por_votaciones[votacion][curp]=posi
            else:
                    por_votaciones[votacion] = {"ID_voto":limpiaVotacion(votacion),"Nombre_Votación":votacion, curp:posi}
            votos['votación']=votacion
            votos['posición']=posi
            aux2['votos']=aux2.get('votos', []) + [votos]
            #dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos', []) + [votos]
        dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos',0)+len(aux2.get('votos', [])) # Se comenta si la de arriba se descomenta


        

In [3]:
def creaCurp(nombre,cumple):
    
    '''
    iniciales = ''.join([c for c in nombre[1:] if c.isupper()])
    c = cumple.replace('-', ' ').split(' ')
    if(len(c[0])==1):
        c[0]="0"+c[0] 
    curp = iniciales[0:3]+c[0]+c[1][0:3]+c[2][-2:]
    '''
    curp = nombre.replace('.', '')
    curp = curp.replace(' ', '')
    curp = curp.upper()
    curp = curp.replace('Á', 'A')
    curp = curp.replace('É', 'E')
    curp = curp.replace('Í', 'I')
    curp = curp.replace('Ó', 'O')
    curp = curp.replace('Ú', 'U')
    curp = curp.replace('Ñ', 'N')
    return curp

In [4]:
def limpiaText(nom):
    resp = re.sub('\s+',' ',nom)
    resp = resp.replace('Dip. ', '')
    resp = resp.replace('(LICENCIA)', '')
    resp = resp.replace('Suplente: ', '')
    resp = resp.replace('(DECESO)', '')
    resp = resp.replace('(Deceso)', '')
    resp = resp.replace('  ', ' ')
    if(resp!="" and resp[-1]==" "):
        resp = resp[0:len(resp)-1]
    if(resp!="" and resp[0]==" "):
        resp = resp[1:]
    return resp

In [5]:
def limpiaVotacion(nombreVot):
    nombreVot = nombreVot.replace('(', '')
    nombreVot = nombreVot.replace(')', '')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace('DECRETO', '')
    nombreVot = nombreVot.replace(' QUE ', ' ')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace(' DE ', ' ')
    nombreVot = nombreVot.replace(' A ', ' ')
    nombreVot = nombreVot.replace(' LA ', ' ')
    nombreVot = nombreVot.replace(' EL ', ' ')
    nombreVot = nombreVot.replace(' POR ', ' ')
    nombreVot = nombreVot.replace(' PARA ', ' ')
    nombreVot = nombreVot.replace(' LOS ', ' ')
    nombreVot = nombreVot.replace(' LAS ', ' ')
    nombreVot = nombreVot.replace(' Y ', ' ')
    nombreVot = nombreVot.replace(' E ', ' ')
    nombreVot = nombreVot.replace(' CON ', ' ')
    nombreVot = nombreVot.replace(' LEY ', ' ')
    nombreVot = nombreVot.replace(' EN ', ' ')
    nombreVot = nombreVot.replace(' LO ', ' ')
    nombreVot = nombreVot.replace(' DEL ', ' ')
    nombreVot = nombreVot.replace('.', ' ')
    nombreVot = nombreVot.replace(';', ' ')
    nombreVot = nombreVot.replace(',', ' ')
    nombreVot = nombreVot.replace('  ', ' ')
    nombreFin=""
    for s in nombreVot.split():
        nombreFin+=s[0]
    return nombreFin
    

In [6]:
def obtienePartido(imagen):
    res=""
    if(imagen=='images/pan.png'):
        res = "PAN"
    elif(imagen=='images/logpt.jpg'):
        res = "PT"
    elif(imagen=='images/LogoMorena.jpg'):
        res = "Morena"
    elif(imagen=='images/encuentro.png'):
        res = "Encuentro Social"
    elif(imagen=='images/logo_movimiento_ciudadano.png'):
        res = "Movimiento Ciudadano"
    elif(imagen=='images/pri01.png'):
        res = "PRI"
    elif(imagen=='images/logvrd.jpg'):
        res = "Partido Verde"
    elif(imagen=='images/prd01.png'):
        res = "PRD"
    elif(imagen=='images/logo_SP.jpg'):
        res = "Sin Partido"
    elif(imagen=="images/panal.gif"):
        res = "Nueva Alianza"
    elif(imagen=="images/independiente.png"):
        res = "Independiente"
    elif(imagen=="images/pasc.jpg"):
        res = "PSD"
    elif(imagen=="images/convergencia.gif"):
        res = "Convergencia"
    else:
         res = "Sin Partido"
    return res
                                    

In [7]:
dipus={}
curp = "h"
urlDipu = "http://sitllxi.diputados.gob.mx/"  
urlCurr = "http://sitllxi.diputados.gob.mx/curricula.php?dipt="  #MODIFICAR ESTO AL PASAR A 60

por_votaciones = {}

terminaciones = []
for i in range(0,1):
    for j in range(1,2):
        for k in range(1,3):
            terminaciones.append(str(i)+str(j)+str(k))
#terminaciones.append(str(500))
terminaciones.pop(0)


# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-04-09 18:11:49 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-04-09 18:11:49 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-04-09 18:11:49 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-04-09 18:11:49 [scrapy.crawler] INFO: Overridden settings:
{}
2021-04-09 18:11:49 [scrapy.extensions.telnet] INFO: Telnet Password: bc35096b0d15c58e
2021-04-09 18:11:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-04-09 18:11:50 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

In [8]:
list_of_dipus = [value for value in dipus.values()]
list_of_dipus

[{'CURP': 'JUANADOLFOORCIMARTINEZ',
  'Nombre': 'Juan Adolfo Orcí Martínez',
  'TipoElección': '\t\t\t\t\t\t\t\t\t    ',
  'Entidad': '\n\t\t\t\t\t\t\t\t\t     ',
  'Partido': 'PRD',
  'Distrito': 'Mayoría Relativa 1 ',
  'Correo': 'juan.orci@congreso.gob.mx ',
  'Onomástico': 'Mulege',
  'Suplente': 'Fidencio Perpuli Pagés',
  'Comisiones': [' Pesca (Secretaría) ',
   ' Presupuesto y Cuenta Pública  ',
   ' Turismo (Secretaría) ',
   ' Citricultura (Secretaría) '],
  'Último grado de estudios': '\t\t\t\t\t\t\t\t\t    ',
  'votos': 590}]

In [9]:
list_of_votos = [value for value in por_votaciones.values()]
list_of_votos

In [10]:
with open('diputados_61.json', 'a') as f:      #MODIFICAR ESTO AL PASAR A 60
            json.dump(list_of_dipus, f, indent=4, ensure_ascii=False) # antes dipus

In [11]:
with open('votaciones_61.json', 'a') as f:     #MODIFICAR ESTO AL PASAR A 60
            json.dump(list_of_votos, f, indent=4, ensure_ascii=False)

In [10]:
nom = ' \n\n\n                          12-octubre\n                          \n\n\n                          '
limpiaText(nom)

'12-octubre'

In [43]:
limpiaText("De. lfino (LICENCIA) López Aparicio (DECESO) ")

'De. lfino López Aparicio'

In [25]:
creaCurp('De. lfino López Aparicio',"")

'DELFINOLOPEZAPARICIO'